# MetaTrader: Analysis multiple markets

This example will get all markets from MetaTrader account and analysis by a simple strategy

## MetaTrader API

Load MetaTrader environment variable from `.env` file.

Follow [tutorial](https://awesometrading.github.io/LetTrade/install/metatrader/) to install MetaTrader


```bash
# .env file
MT5_LOGIN=13456789
MT5_PASSWORD=P@ssW0rd
MT5_SERVER=RoboForex

MT5_WINE=WINEPREFIX=$HOME/.mt5 python -m mt5linux "$HOME/.mt5/dosdevices/c:/users/$USER/AppData/Local/Programs/Python/Python310-32/python.exe"
```

In [1]:
from dotenv import load_dotenv
import os
from lettrade.exchange.metatrader import MetaTraderAPI

load_dotenv()

api = MetaTraderAPI(
    login=int(os.environ["MT5_LOGIN"]),
    password=os.environ["MT5_PASSWORD"],
    server=os.environ["MT5_SERVER"],
    wine=os.getenv("MT5_WINE", None),
)


028c:fixme:file:server_get_file_info Unsupported info class e
028c:fixme:file:server_get_file_info Unsupported info class e
028c:err:winediag:ntlm_check_version ntlm_auth was not found. Make sure that ntlm_auth >= 3.0.25 is in your path. Usually, you can find it in the winbind package of your distribution.
028c:err:ntlm:ntlm_LsaApInitializePackage no NTLM support, expect problems
Oct 26 19:38:51 INFO SLAVE/18812[MainThread]: server started on [127.0.0.1]:18812
Oct 26 19:38:55 INFO SLAVE/18812[MainThread]: accepted ('127.0.0.1', 59774) with fd 152
Oct 26 19:38:55 INFO SLAVE/18812[Thread-1 (_authenticate_and_serve_client)]: welcome ('127.0.0.1', 59774)
02bc:fixme:uxtheme:SetPreferredAppMode 0: stub
02bc:fixme:ntdll:NtQuerySystemInformation info_class SYSTEM_PERFORMANCE_INFORMATION
02c0:fixme:kernelbase:AppPolicyGetThreadInitializationType FFFFFFFFFFFFFFFA, 0000000001BBFF50
02d8:fixme:uxtheme:SetPreferredAppMode 0: stub
02e4:fixme:kernelbase:AppPolicyGetThreadInitializationType FFFFFFFFFF

## Load markets

Load markets information from MetaTraderAPI

In [2]:
markets = api.markets("*")

### Filter markets

Filter some type of market

In [3]:
filtered_markets = []
for market in markets:
    if market.type not in ["Forex", "Stocks", "CFD"]:
        continue
    filtered_markets.append(market)

## Strategy

Simple strategy will check:

- Buy signal:
    + `1w` timeframe: `close` > `EMA9`
    + `1d` timeframe: `close` > `EMA21` and `EMA9` > `EMA21`

- Sell signal:
    + `1w` timeframe: `close` < `EMA9`
    + `1d` timeframe: `close` < `EMA21` and `EMA9` < `EMA21`


In [4]:
import logging

from lettrade import DataFeed, Strategy

logger = logging.getLogger(__name__)


class SimpleStrategy(Strategy):
    # Params
    ema_fast_window: int = 9
    ema_main_window: int = 21

    # Information
    symbol: str

    def indicators(self, df: DataFeed):
        df.i.ema(window=self.ema_fast_window, name="ema_fast", inplace=True)
        df.i.ema(window=self.ema_main_window, name="ema_main", inplace=True)

        df.i.rolling_direction(
            "close",
            "ema_main",
            window=3,
            name="close_rolling_one_side_ema_main",
            inplace=True,
        )

        df.i.rolling_direction(
            "ema_fast",
            "ema_main",
            window=3,
            name="ema_fast_rolling_one_side_ema_main",
            inplace=True,
        )

        # Signal
        df.i.signal_direction(
            # --- Buy Signal
            (df["close_rolling_one_side_ema_main"] >= 100)
            & (df["ema_fast_rolling_one_side_ema_main"] >= 100),
            # --- Sell Signal
            (df["close_rolling_one_side_ema_main"] <= -100)
            & (df["ema_fast_rolling_one_side_ema_main"] <= -100),
            name="signal",
            inplace=True,
        )

    # Only load indicator for `data_high` datafeed
    def indicators_data_high(self, df: DataFeed):
        df.i.ema(window=self.ema_fast_window, name="ema_fast", inplace=True)

        df.i.rolling_direction(
            "close",
            "ema_fast",
            window=2,
            name="close_rolling_one_side_ema_fast",
            inplace=True,
        )

        # Signal
        df.i.signal_direction(
            # --- Buy Signal
            (df["close_rolling_one_side_ema_fast"] >= 100),
            # --- Sell Signal
            (df["close_rolling_one_side_ema_fast"] <= -100),
            name="signal",
            inplace=True,
        )

    def next(
        self,
        df_low: DataFeed,
        df_high: DataFeed,
    ):
        if df_low.l.signal[0] >= 100 and df_high.l.signal[0] >= 100:
            self.send_notify("BUY", symbol=self.symbol)
        if df_low.l.signal[0] <= -100 and df_high.l.signal[0] <= -100:
            self.send_notify("SELL", symbol=self.symbol)

### Prepare commander

`StorageBackTestCommander` will store all notify from strategy 

In [5]:
from lettrade.exchange.backtest import StorageBackTestCommander

commander = StorageBackTestCommander()

## Run

Test markets by download data from MetaTrader and run strategy

In [6]:
from lettrade.exchange.backtest import (
    ForexBackTestAccount,
    BackTestDataFeed,
    let_backtest,
)
from lettrade.exchange.metatrader import MetaTraderDataFeed


def run_backtest(symbol):
    data_low = MetaTraderDataFeed(symbol=symbol, timeframe="1d", api=api)
    data_low.bars_load(since=0, to=1000)
    data_high = MetaTraderDataFeed(symbol=symbol, timeframe="1w", api=api)
    data_high.bars_load(since=0, to=100)

    lt = let_backtest(
        strategy=SimpleStrategy,
        datas=[
            # Wrap to BacktestDataFeed type
            BackTestDataFeed(data_low, name="data_low"),
            BackTestDataFeed(data_high, name="data_high"),
        ],
        account=ForexBackTestAccount,
        commander=commander,
        # Send `symbol` params to strategy
        strategy_kwargs=dict(
            symbol=symbol,
        ),
    )

    # Only run last bar
    lt.run(start_kwargs=dict(feed_size=len(data_low) - 2))


for market in filtered_markets[0:10]:
    run_backtest(market["symbol"])

# Strategy                <class '__main__.SimpleStrategy'>
Start                             2020-11-04 00:00:00+00:00
End                               2024-10-25 00:00:00+00:00
Duration                                 1451 days 00:00:00
Start Balance                                         10000
Equity [$]                                            10000
Equity Peak [$]                                       10000
PL [$]                                                    0
PL [%]                                                  0.0
Buy & Hold PL [%]                                     18.69
Max. Drawdown [%]                                      -0.0
Avg. Drawdown [%]                                       NaN
Max. Drawdown Duration                                  NaN
Avg. Drawdown Duration                                  NaN
                                                           
# Positions                                               0
Win Rate [%]                            

## Result

Storage is notify result of strategies

In [7]:
commander.storage

[{'message': 'BUY', 'symbol': 'AAL'},
 {'message': 'SELL', 'symbol': 'AAP'},
 {'message': 'BUY', 'symbol': 'AAPL'},
 {'message': 'SELL', 'symbol': 'ABBV'},
 {'message': 'BUY', 'symbol': 'ABNB'},
 {'message': 'SELL', 'symbol': 'ACAD'},
 {'message': 'SELL', 'symbol': 'ACGL'}]